In [1]:
from dotenv import load_dotenv
import requests
import os
import pandas as pd
import overpy

In [3]:
api = overpy.Overpass()

In [5]:
query = """
[out:json];
area[name="New York"]->.searchArea;
(
  node["amenity"="bar"](area.searchArea);
  node["amenity"="biergarten"](area.searchArea);
  node["amenity"="cafe"](area.searchArea);
 
);
out center;
"""

In [6]:
result = api.query(query)

In [12]:
for p in dir(result.nodes[0]):
    print(p)

__annotations__
__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__getstate__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__
__weakref__
_result
_type_value
attributes
from_json
from_xml
get_center_from_json
get_center_from_xml_dom
id
lat
lon
tags


In [35]:
print(set(result.nodes[0].tags.keys()))

{'cuisine', 'name', 'addr:street', 'amenity', 'website', 'addr:housenumber', 'addr:postcode'}


In [40]:
params = set()

for node in result.nodes:
    print(set(node.tags.keys()))
    params = params.union(set(node.tags.keys()))

print(params)

{'cuisine', 'name', 'addr:street', 'amenity', 'website', 'addr:housenumber', 'addr:postcode'}
{'amenity', 'name'}
{'cuisine', 'name', 'addr:street', 'amenity', 'website', 'addr:housenumber', 'addr:postcode'}
{'amenity', 'name'}
{'cuisine', 'addr:city', 'name', 'addr:state', 'addr:street', 'amenity', 'phone', 'website', 'addr:housenumber', 'addr:postcode'}
{'cuisine', 'addr:city', 'name', 'addr:state', 'addr:street', 'amenity', 'phone', 'website', 'addr:housenumber', 'addr:postcode'}
{'created_by', 'amenity', 'name'}
{'cuisine', 'addr:city', 'name', 'nysgissam:nysaddresspointid', 'addr:state', 'addr:street', 'amenity', 'addr:housenumber', 'addr:postcode'}
{'amenity', 'name'}
{'cuisine', 'addr:city', 'name', 'addr:state', 'addr:street', 'amenity', 'phone', 'website', 'addr:housenumber', 'addr:postcode'}
{'created_by', 'amenity', 'name'}
{'amenity', 'name'}
{'cuisine', 'wheelchair', 'air_conditioning', 'indoor_seating', 'name', 'amenity', 'outdoor_seating', 'smoking'}
{'created_by', 'amen

In [18]:
print(len(result.nodes))

10722


In [ ]:
cols = 